In [ ]:
# Step 1: Upload and Load Data
from google.colab import files
uploaded = files.upload()

Saving js_model.pt to js_model.pt


In [ ]:


import gzip
import json

data = []
for i in range(5):
    with gzip.open(f"javascript_train_{i}.jsonl.gz", "rt", encoding="utf-8") as f:
      for line in f:
          obj = json.loads(line)
          code = obj.get("code", "").strip()
          if code:
              data.append(code)

print(f"Loaded {len(data)} code snippets.")
print(data[0][:300])  # preview

# Step 2: Tokenize using tiktoken

import tiktoken
enc = tiktoken.get_encoding("gpt2")

tokens = []
for code in data:
    tokens.extend(enc.encode(code))

import torch
tokens = torch.tensor(tokens, dtype=torch.long)
train_data = tokens[:int(0.9 * len(tokens))]
val_data = tokens[int(0.9 * len(tokens)):]

# Step 3: Batching logic
block_size = 128
batch_size = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x.to(device), y.to(device)

# Step 4: Transformer model definition
import torch.nn as nn
import torch.nn.functional as F
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)

class MultiHeadAttention(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        self.n_head = n_head
        self.head_size = n_embd // n_head
        self.qkv = nn.Linear(n_embd, 3 * n_embd)
        self.proj = nn.Linear(n_embd, n_embd)

    def forward(self, x):
        B, T, C = x.shape
        qkv = self.qkv(x).view(B, T, self.n_head, 3 * self.head_size).transpose(1, 2)
        q, k, v = qkv.chunk(3, dim=-1)

        attn_scores = (q @ k.transpose(-2, -1)) * (self.head_size ** -0.5)
        mask = torch.tril(torch.ones(T, T, device=x.device)).unsqueeze(0).unsqueeze(0)
        attn_scores = attn_scores.masked_fill(mask == 0, float('-inf'))
        attn = F.softmax(attn_scores, dim=-1)

        out = attn @ v
        out = out.transpose(1, 2).contiguous().view(B, T, C)
        return self.proj(out)

class FeedForward(nn.Module):
    def __init__(self, n_embd, d_ff):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, n_embd),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head, d_ff):
        super().__init__()
        self.attn = MultiHeadAttention(n_embd, n_head)
        self.ff = FeedForward(n_embd, d_ff)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model, n_head, d_ff, n_layer, block_size):
        super().__init__()
        self.block_size = block_size
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model)
        self.blocks = nn.ModuleList([
            Block(d_model, n_head, d_ff) for _ in range(n_layer)
        ])
        self.norm = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size)

    def forward(self, x, targets=None):
        x = self.token_emb(x)
        x = self.pos_enc(x)
        for block in self.blocks:
            x = block(x)
        x = self.norm(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_token), dim=1)
        return idx

# Step 5: Model Initialization
vocab_size = enc.n_vocab
model = Transformer(vocab_size, d_model=256, n_head=4, d_ff=1024, n_layer=4, block_size=block_size).to(device)

# Step 6: Training
max_iters = 1000
eval_interval = 200
eval_iters = 20
learning_rate = 1e-3
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(split)
            _, loss = model(xb, yb)
            losses[k] = loss.item()
        out[split] = losses.mean().item()
    model.train()
    return out

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Step 7: Generate Code
context = torch.zeros((1, 1), dtype=torch.long).to(device)  # BOS token or start of context
output = model.generate(context, max_new_tokens=200)
print(enc.decode(output[0].tolist()))


Loaded 123889 code snippets.
function createTypeScriptLanguageService(options) {
    // Discover referenced files
    const FILES = discoverAndReadFiles(options);
    // Add fake usage files
    options.inlineEntryPoints.forEach((inlineEntryPoint, index) => {
        FILES[`inlineEntryPoint.${index}.ts`] = inlineEntryPoint;
   
step 0: train loss 10.9358, val loss 10.9452
step 200: train loss 3.1363, val loss 3.2670
step 400: train loss 2.8535, val loss 3.0104
step 600: train loss 2.6741, val loss 2.8578
step 800: train loss 2.5689, val loss 2.6966
step 999: train loss 2.4646, val loss 2.6309
!githubSetGrid', 'option'])
         glyphDistafWatch': this.draw1
         wrapper.menu = this;
       }function |version(value) {
        this.map.charAt(0,{"0)|0)/ phase,
          = [];
    }
      this.ok(enendServices.DD, "";
       gl.dirtyGW++;
      }
    }

lim
  function ensureAsegment.addLine 24Lum(event)AM 398
     ; = me.each(0, data.body);
  


In [ ]:
# Damn thats worst output :(

In [ ]:
max_iters = 5000
eval_interval = 500
eval_iters = 20
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

step 0: train loss 2.4721, val loss 2.6887
step 500: train loss 2.2813, val loss 2.5202
step 1000: train loss 2.1846, val loss 2.4043
step 1500: train loss 2.0603, val loss 2.2678
step 2000: train loss 2.0285, val loss 2.3163
step 2500: train loss 1.9920, val loss 2.2856
step 3000: train loss 1.9648, val loss 2.2177
step 3500: train loss 1.8996, val loss 2.1656
step 4000: train loss 1.8547, val loss 2.1236
step 4500: train loss 1.8032, val loss 2.1330
step 4999: train loss 1.8137, val loss 2.0508


In [ ]:
context = torch.zeros((1, 1), dtype=torch.long).to(device)  # BOS token or start of context
output = model.generate(context, max_new_tokens=200)
print(enc.decode(output[0].tolist()))

!picker(d), ranges / 180Filters.addClass(elementsBy.CLASSins.disabled)*values.midSegmentCore.kill.isaSlug),
					time = allDialog.datatableRegions.gasizeDetectorHape({}, settingsByTODES.api.position)
					}).join(",groupName).map(V => {
           var self = this[esByHOUREqual(iPodFnPropertyToken),
	             values = columns.subnumericIndexOfSlrQRTLim(iVBatorsCommaFirstTokenEquals, name));
           var possibleId = models[i];

          if (
       


In [ ]:
# magic <_<


In [ ]:
max_iters = 20000
eval_interval = 500
eval_iters = 20
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

step 0: train loss 1.7834, val loss 2.0846
step 500: train loss 1.7990, val loss 1.9742
step 1000: train loss 1.7302, val loss 2.0430
step 1500: train loss 1.8141, val loss 1.9653
step 2000: train loss 1.7216, val loss 1.9607
step 2500: train loss 1.7387, val loss 1.9547
step 3000: train loss 1.7211, val loss 1.9730


In [ ]:
torch.save(model.state_dict(), "js_model.pt")


In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:

import gzip
import json

data = []
for i in range(5):
    with gzip.open(f"javascript_train_{i}.jsonl.gz", "rt", encoding="utf-8") as f:
      for line in f:
          obj = json.loads(line)
          code = obj.get("code", "").strip()
          if code:
              data.append(code)

print(f"Loaded {len(data)} code snippets.")
print(data[0][:300])  # preview

# Step 2: Tokenize using tiktoken

import tiktoken
enc = tiktoken.get_encoding("gpt2")

tokens = []
for code in data:
    tokens.extend(enc.encode(code))

import torch
tokens = torch.tensor(tokens, dtype=torch.long)
train_data = tokens[:int(0.9 * len(tokens))]
val_data = tokens[int(0.9 * len(tokens)):]

# Step 3: Batching logic
block_size = 128
batch_size = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x.to(device), y.to(device)

Loaded 123889 code snippets.
function createTypeScriptLanguageService(options) {
    // Discover referenced files
    const FILES = discoverAndReadFiles(options);
    // Add fake usage files
    options.inlineEntryPoints.forEach((inlineEntryPoint, index) => {
        FILES[`inlineEntryPoint.${index}.ts`] = inlineEntryPoint;
   


In [ ]:
# Step 4: Transformer model definition
import torch.nn as nn
import torch.nn.functional as F
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)

class MultiHeadAttention(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        self.n_head = n_head
        self.head_size = n_embd // n_head
        self.qkv = nn.Linear(n_embd, 3 * n_embd)
        self.proj = nn.Linear(n_embd, n_embd)

    def forward(self, x):
        B, T, C = x.shape
        qkv = self.qkv(x).view(B, T, self.n_head, 3 * self.head_size).transpose(1, 2)
        q, k, v = qkv.chunk(3, dim=-1)

        attn_scores = (q @ k.transpose(-2, -1)) * (self.head_size ** -0.5)
        mask = torch.tril(torch.ones(T, T, device=x.device)).unsqueeze(0).unsqueeze(0)
        attn_scores = attn_scores.masked_fill(mask == 0, float('-inf'))
        attn = F.softmax(attn_scores, dim=-1)

        out = attn @ v
        out = out.transpose(1, 2).contiguous().view(B, T, C)
        return self.proj(out)

class FeedForward(nn.Module):
    def __init__(self, n_embd, d_ff):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, n_embd),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head, d_ff):
        super().__init__()
        self.attn = MultiHeadAttention(n_embd, n_head)
        self.ff = FeedForward(n_embd, d_ff)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model, n_head, d_ff, n_layer, block_size):
        super().__init__()
        self.block_size = block_size
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model)
        self.blocks = nn.ModuleList([
            Block(d_model, n_head, d_ff) for _ in range(n_layer)
        ])
        self.norm = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size)

    def forward(self, x, targets=None):
        x = self.token_emb(x)
        x = self.pos_enc(x)
        for block in self.blocks:
            x = block(x)
        x = self.norm(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_token), dim=1)
        return idx

In [ ]:
vocab_size = enc.n_vocab
model = Transformer(vocab_size, d_model=256, n_head=4, d_ff=1024, n_layer=4, block_size=128).to(device)
model.load_state_dict(torch.load("js_model.pt", map_location=device))
model.eval()

Transformer(
  (token_emb): Embedding(50257, 256)
  (pos_enc): PositionalEncoding()
  (blocks): ModuleList(
    (0-3): 4 x Block(
      (attn): MultiHeadAttention(
        (qkv): Linear(in_features=256, out_features=768, bias=True)
        (proj): Linear(in_features=256, out_features=256, bias=True)
      )
      (ff): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=256, out_features=1024, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1024, out_features=256, bias=True)
        )
      )
      (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
  )
  (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (lm_head): Linear(in_features=256, out_features=50257, bias=True)
)

In [ ]:
# Half-written JavaScript code prompt
prompt = """
function sum(a, b) {
  return
"""

# Encode the prompt using your tokenizer
input_ids = torch.tensor(enc.encode(prompt), dtype=torch.long).unsqueeze(0).to(device)  # shape: [1, T]

# Generate continuation
output = model.generate(input_ids, max_new_tokens=100)

# Decode and print result
print(enc.decode(output[0].tolist()))



function sum(a, b) {
  return
    / beautify #\s*    kBottom of format( |1974):\/\/-[negative Lookup>"
             +
                   | " +
                    tp(a, rNode, y),
      


In [ ]:
import gzip
import json
import re

# Input/output
output_path = "clean_code.jsonl"

def is_clean_code(code):
    if len(code) < 50 or len(code) > 5000:
        return False
    if code.count('\n') < 2:
        return False
    if "eval(" in code or "Function(" in code or "document.write" in code:
        return False
    if re.search(r"\b[a-zA-Z]{1,2}\b", code):
        short_vars = re.findall(r"\b[a-zA-Z]{1,2}\b", code)
        if len(short_vars) > 15:
            return False
    if re.search(r"[\u0000-\u001f]", code):  # control characters
        return False
    return True

def normalize_code(code):
    return "\n".join(
        line.strip().replace('\t', '   ')
        for line in code.strip().splitlines()
        if line.strip()
    )

# Clean across all files
clean_count = 0
with open(output_path, "w", encoding="utf-8") as f_out:
    for i in range(5):
        fname = f"javascript_train_{i}.jsonl.gz"
        try:
            with gzip.open(fname, "rt", encoding="utf-8") as f_in:
                for line in f_in:
                    obj = json.loads(line)
                    code = obj.get("code", "").strip()
                    if is_clean_code(code):
                        clean_code = normalize_code(code)
                        json.dump({"code": clean_code}, f_out)
                        f_out.write("\n")
                        clean_count += 1
        except FileNotFoundError:
            print(f"⚠️  File not found: {fname}, skipping.")

print(f"✅ Done. Saved {clean_count} clean code snippets to {output_path}")


✅ Done. Saved 0 clean code snippets to clean_code.jsonl


In [ ]:
with open("clean_code.jsonl", "r") as f:
    data = [json.loads(line)["code"] for line in f]


In [ ]:
data[:100]

[]

In [ ]:
import gzip
import json
import re

# Output file
output_path = "clean_code.jsonl"

# Heuristics for good code
def is_clean_code(code, docstring):
    if not code or len(code) < 30 or len(code) > 3000:
        return False
    if "eval(" in code or "Function(" in code or "document.write" in code:
        return False
    if code.count('\n') < 1:
        return False
    return True

# Normalization (light cleanup)
def normalize_code(code):
    return "\n".join(
        line.strip().replace('\t', '    ')
        for line in code.strip().splitlines()
        if line.strip()
    )

clean_count = 0

with open(output_path, "w", encoding="utf-8") as f_out:
    for i in range(5):
        fname = f"javascript_train_{i}.jsonl.gz"
        try:
            with gzip.open(fname, "rt", encoding="utf-8") as f_in:
                for line in f_in:
                    try:
                        obj = json.loads(line)
                        code = obj.get("code", "").strip()
                        docstring = obj.get("docstring", "").strip()
                        if is_clean_code(code, docstring):
                            clean_code = normalize_code(code)
                            json.dump({"code": clean_code, "docstring": docstring}, f_out)
                            f_out.write("\n")
                            clean_count += 1
                    except json.JSONDecodeError:
                        continue
        except FileNotFoundError:
            continue

print(f"✅ Done. Saved {clean_count} clean code+docstring pairs to {output_path}")


✅ Done. Saved 116120 clean code+docstring pairs to clean_code.jsonl


In [ ]:
with open("clean_code.jsonl", "r") as f:
    data = [json.loads(line)["code"] for line in f]

In [ ]:
data[:1]

['function createTypeScriptLanguageService ( options ) { / / Discover referenced files const FILES = discoverAndReadFiles ( options ) ; / / Add fake usage files options . inlineEntryPoints . forEach ( ( inlineEntryPoint , index ) = > { FILES [ ` inlineEntryPoint . $ { index } . ts ` ] = inlineEntryPoint ; } ) ; / / Add additional typings options . typings . forEach ( ( typing ) = > { const filePath = path . join ( options . sourcesRoot , typing ) ; FILES [ typing ] = fs . readFileSync ( filePath ) . toString ( ) ; } ) ; / / Resolve libs const RESOLVED _ LIBS = { } ; options . libs . forEach ( ( filename ) = > { const filepath = path . join ( TYPESCRIPT _ LIB _ FOLDER , filename ) ; RESOLVED _ LIBS [ ` defaultLib : $ { filename } ` ] = fs . readFileSync ( filepath ) . toString ( ) ; } ) ; const compilerOptions = ts . convertCompilerOptionsFromJson ( options . compilerOptions , options . sourcesRoot ) . options ; const host = new TypeScriptLanguageServiceHost ( RESOLVED _ LIBS , FILES , 

In [ ]:
import gzip
import json
import re
import os

# ========== Normalization Patterns ==========
normalize1 = [
    ('<skipped>', ''),
    (r'-\n', ''),
    (r'\n', ' ')
]
normalize1 = [(re.compile(pattern), repl) for pattern, repl in normalize1]

normalize2 = [
    (r'([\{-\~\[-\` -\&\(-\+\:-\@\/])', r' \1 '),
    (r'([^0-9])([\.,])', r'\1 \2 '),
    (r'([\.,])([^0-9])', r' \1 \2'),
    (r'([0-9])(-)', r'\1 \2 ')
]
normalize2 = [(re.compile(pattern), repl) for pattern, repl in normalize2]

def normalize_code(code):
    code = code.strip()
    for pattern, repl in normalize1:
        code = pattern.sub(repl, code)
    for pattern, repl in normalize2:
        code = pattern.sub(repl, code)
    return ' '.join(code.split())  # remove redundant spaces

# ========== Optional: Filtering Function ==========
def is_clean_code(code):
    if len(code) < 10: return False
    if 'eval' in code or 'base64' in code: return False
    if len(code.splitlines()) < 2: return False
    if len(set(code)) < 10: return False
    return True

# ========== Output Path ==========
output_path = "clean_code.jsonl"

# ========== Clean & Save ==========
clean_count = 0
with open(output_path, "w", encoding="utf-8") as f_out:
    for i in range(5):
        input_path = f"javascript_train_{i}.jsonl.gz"
        if not os.path.exists(input_path):
            print(f"⚠️ File not found: {input_path}")
            continue
        with gzip.open(input_path, "rt", encoding="utf-8") as f_in:
            for line in f_in:
                try:
                    obj = json.loads(line)
                    code = obj.get("code", "").strip()
                    if is_clean_code(code):
                        clean_code = normalize_code(code)
                        json.dump({"code": clean_code}, f_out)
                        f_out.write("\n")
                        clean_count += 1
                except Exception as e:
                    continue  # skip bad lines

print(f"✅ Done. Saved {clean_count} clean code snippets to {output_path}")


✅ Done. Saved 122113 clean code snippets to clean_code.jsonl


In [ ]:
import torch

RuntimeError: Trying to override a python impl for DispatchKey.Meta on operator aten::broadcast_tensors

In [ ]:
import json
import torch
import tiktoken

# Load tokenizer
enc = tiktoken.get_encoding("gpt2")

# Load cleaned JSONL file
data = []
with open("clean_code.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        code = obj.get("code", "")
        if code:
            data.append(code)

print(f"✅ Loaded {len(data)} cleaned code snippets.")

# Tokenize all code snippets into a flat list of token IDs
all_tokens = []
for snippet in data:
    all_tokens.extend(enc.encode(snippet))

print(f"✅ Total tokens: {len(all_tokens)}")

# Convert tokens to a torch tensor
tokens_tensor = torch.tensor(all_tokens, dtype=torch.long)

# Train/val split (90/10)
split_idx = int(0.9 * len(tokens_tensor))
train_data = tokens_tensor[:split_idx]
val_data = tokens_tensor[split_idx:]

print(f"Train shape: {train_data.shape}, Val shape: {val_data.shape}")

# ========== Batch Sampling Function ==========
block_size = 128
batch_size = 32

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(0, len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x.to(device), y.to(device)

# Quick test
xb, yb = get_batch("train")
print("Sample input decoded:", enc.decode(xb[0].tolist()))
print("Sample target decoded:", enc.decode(yb[0].tolist()))


RuntimeError: Trying to override a python impl for DispatchKey.Meta on operator aten::broadcast_tensors

In [ ]:
data[:1]

['function createTypeScriptLanguageService ( options ) { / / Discover referenced files const FILES = discoverAndReadFiles ( options ) ; / / Add fake usage files options . inlineEntryPoints . forEach ( ( inlineEntryPoint , index ) = > { FILES [ ` inlineEntryPoint . $ { index } . ts ` ] = inlineEntryPoint ; } ) ; / / Add additional typings options . typings . forEach ( ( typing ) = > { const filePath = path . join ( options . sourcesRoot , typing ) ; FILES [ typing ] = fs . readFileSync ( filePath ) . toString ( ) ; } ) ; / / Resolve libs const RESOLVED _ LIBS = { } ; options . libs . forEach ( ( filename ) = > { const filepath = path . join ( TYPESCRIPT _ LIB _ FOLDER , filename ) ; RESOLVED _ LIBS [ ` defaultLib : $ { filename } ` ] = fs . readFileSync ( filepath ) . toString ( ) ; } ) ; const compilerOptions = ts . convertCompilerOptionsFromJson ( options . compilerOptions , options . sourcesRoot ) . options ; const host = new TypeScriptLanguageServiceHost ( RESOLVED _ LIBS , FILES , 

In [ ]:
# Step 4: Transformer model definition
import torch.nn as nn
import torch.nn.functional as F
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)

class MultiHeadAttention(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        self.n_head = n_head
        self.head_size = n_embd // n_head
        self.qkv = nn.Linear(n_embd, 3 * n_embd)
        self.proj = nn.Linear(n_embd, n_embd)

    def forward(self, x):
        B, T, C = x.shape
        qkv = self.qkv(x).view(B, T, self.n_head, 3 * self.head_size).transpose(1, 2)
        q, k, v = qkv.chunk(3, dim=-1)

        attn_scores = (q @ k.transpose(-2, -1)) * (self.head_size ** -0.5)
        mask = torch.tril(torch.ones(T, T, device=x.device)).unsqueeze(0).unsqueeze(0)
        attn_scores = attn_scores.masked_fill(mask == 0, float('-inf'))
        attn = F.softmax(attn_scores, dim=-1)

        out = attn @ v
        out = out.transpose(1, 2).contiguous().view(B, T, C)
        return self.proj(out)

class FeedForward(nn.Module):
    def __init__(self, n_embd, d_ff):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, n_embd),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head, d_ff):
        super().__init__()
        self.attn = MultiHeadAttention(n_embd, n_head)
        self.ff = FeedForward(n_embd, d_ff)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model, n_head, d_ff, n_layer, block_size):
        super().__init__()
        self.block_size = block_size
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model)
        self.blocks = nn.ModuleList([
            Block(d_model, n_head, d_ff) for _ in range(n_layer)
        ])
        self.norm = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size)

    def forward(self, x, targets=None):
        x = self.token_emb(x)
        x = self.pos_enc(x)
        for block in self.blocks:
            x = block(x)
        x = self.norm(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_token), dim=1)
        return idx